In [15]:
import time
from getpass import getpass
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException,\
                    ElementClickInterceptedException, ElementNotInteractableException,TimeoutException

# Login and search info ...
email = input("Enter your Email: ")
# getpass module for make entering password secure ...
password = getpass("Enter your password: ")
category_search = input("Enter The Category: ")
Location_search = input("Enter The Location: ")

PATH = "C:\Program Files (x86)\chromedriver.exe"
driver = webdriver.Chrome(PATH)
driver.maximize_window()

ignored_exceptions=(NoSuchElementException,StaleElementReferenceException,\
                    ElementClickInterceptedException, ElementNotInteractableException,TimeoutException)

driver.get("https://www.linkedin.com/")
driver.implicitly_wait(10)

# Send the login and Search info
driver.find_element(By.XPATH,'//input[@autocomplete="username"]').send_keys(email)
driver.find_element(By.XPATH,'//input[@autocomplete="current-password"]').send_keys(password)
driver.find_element(By.XPATH,'//button[@class="sign-in-form__submit-button"]').click()
time.sleep(5)
category = WebDriverWait(driver, 15,ignored_exceptions=ignored_exceptions)\
            .until(EC.presence_of_element_located\
                ((By.XPATH, '//input[@class="search-global-typeahead__input always-show-placeholder"]')))

category.send_keys(category_search)
time.sleep(3)
category.send_keys(Keys.ENTER)
time.sleep(3)
WebDriverWait(driver, 30,ignored_exceptions=ignored_exceptions)\
            .until(EC.presence_of_element_located\
                ((By.XPATH, '//button[@aria-label="Companies"]'))).click()
WebDriverWait(driver, 30,ignored_exceptions=ignored_exceptions)\
            .until(EC.presence_of_element_located\
                ((By.XPATH, '//button[text()="Locations"]'))).click()
WebDriverWait(driver, 30,ignored_exceptions=ignored_exceptions)\
            .until(EC.presence_of_element_located\
                ((By.XPATH, '//input[@placeholder="Add a location"]'))).send_keys(Location_search)
time.sleep(3)
location = WebDriverWait(driver, 15,ignored_exceptions=ignored_exceptions)\
            .until(EC.presence_of_element_located\
                ((By.XPATH, '(//div[@class="search-typeahead-v2__hit search-typeahead-v2__hit--autocomplete"])[1]//span//span')))
location.click()
submit_filter = WebDriverWait(driver, 15,ignored_exceptions=ignored_exceptions)\
            .until(EC.presence_of_element_located\
                ((By.XPATH, '(//button[@data-control-name="filter_show_results"])[1]')))
submit_filter.click()
time.sleep(3)

# Get first 100 links at Category

company_link = []

count = 0
while count < 11:
    time.sleep(5)
    names_links = driver.find_elements(By.XPATH,'//span[@class="entity-result__title-text t-16"]//a')
    for company in names_links :
        company_link.append(company.get_attribute('href'))
    time.sleep(3)
    driver.execute_script("window.scrollBy(0,document.body.scrollHeight)")
    try :
        WebDriverWait(driver, 15,ignored_exceptions=ignored_exceptions)\
            .until(EC.presence_of_element_located\
                ((By.XPATH, '//span[text()="Next"]'))).click()
    except ignored_exceptions:
        break
    count+=1

Enter your Email: mothfr12@gmail.com
Enter your password: ········
Enter The Category: Networks
Enter The Location: United Arab Emirates


<ipython-input-15-3251fb856c4e>:19: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


In [16]:
# Iterate in links to scrape the data ...
company_data = []
for company in company_link:
    time.sleep(3)
    driver.get(company)
    WebDriverWait(driver, 30,ignored_exceptions=ignored_exceptions)\
            .until(EC.presence_of_element_located\
                ((By.XPATH, '//a[text()="About"]'))).click()
    time.sleep(2)
    name = WebDriverWait(driver, 30,ignored_exceptions=ignored_exceptions)\
            .until(EC.presence_of_element_located\
                ((By.XPATH, '(//span[@dir="ltr"])[1]')))
    
    data = driver.find_elements(By.XPATH,'//dd')
    titles = driver.find_elements(By.XPATH,'//dt')
    
    data_list = []
    data_titels = []
    for title in range(len(titles)):
        xpath_title = '//dt[{}]'.format(title+1)
        title_text = WebDriverWait(driver, 30,ignored_exceptions=ignored_exceptions)\
            .until(EC.presence_of_element_located\
                ((By.XPATH, xpath_title))).text
        data_titels.append(title_text)
    for element in range(len(data)):
        xpath_data = '//dd[{}]'.format(element+1)
        data_text = WebDriverWait(driver, 30,ignored_exceptions=ignored_exceptions)\
            .until(EC.presence_of_element_located\
                ((By.XPATH, xpath_data))).text
        data_list.append(data_text)
    data_list = [ x for x in data_list if "on LinkedIn" not in x ]
    res = {}
    for key in data_titels:
        res["Name"] = name.text
        res["Category"] = category_search
        for value in data_list:
            res[key] = value
            data_list.remove(value)
            break
    company_data.append(res)

In [17]:
# Convert the data to DataFrame and.. So it can be stored on Rleational or None Relational Databases
import pandas as pd
df = pd.DataFrame(company_data)
df

,Name,Category,Website,Phone,Company size,Headquarters,Founded,Specialties
0,"Attivo Networks, Inc.",Networks,http://www.attivonetworks.com,5106231000\nPhone number is 5106231000,201-500 employees,"Fremont, CA",2011,"Breach Detection System, Deception Technology ..."
1,Nozomi Networks,Networks,http://www.nozominetworks.com,+1 800 314 6114\nPhone number is +1 800 314 6114,51-200 employees,"San Francisco, California",2013,"Cybersecurity, OT Security, IoT Security, OT C..."
2,Tejas Networks,Networks,http://www.tejasnetworks.com,NaN,"501-1,000 employees","Bangalore, Karnataka",2000,"optical networking, optical transmission & swi..."
3,Infracom Communication Networks,Networks,http://infracommunication.com/,NaN,11-50 employees,"Jebel Ali, Dubai",2008,"TELECOMMUNICATION INFRASTRUCTURE, SECURITY SUR..."
4,March Networks,Networks,https://www.marchnetworks.com,NaN,201-500 employees,"Ottawa, Ontario",2000,"IP Video Surveillance Solutions, High-Definiti..."
...,...,...,...,...,...,...,...,...
105,InnoView - All in one reporting tool for Ad Ne...,Networks,http://www.innoview.me/,NaN,2-10 employees,"Dubai Silicon Oasis , Dubai",2018,NaN
106,BRIELKEYNTS CONSULTING NETWORKS,Networks,NaN,NaN,11-50 employees,"Al Qadsia, Sharjah",2005,"Branding, Management, Precious Minerals, Gold ..."
107,247 Tech - 24 Networks Global,Networks,http://www.247-africa.tech/,NaN,11-50 employees,"Nairobi, Nairobi",2017,"Cybersecurity , Telecommunications , Internet,..."
108,Dexa Networks,Networks,http://www.dexa-ceases-to-exit-now.com,NaN,2-10 employees,"Dubai, AE",2003,NaN


In [18]:
# Store the data in mongodb ...
import pandas as pd
import pymongo
import json

client = pymongo.MongoClient("mongodb://localhost:27017")
data = df.to_dict(orient = "records")
db = client["Networks_Companies"]
db.Iris.insert_many(data)

<img src="Capture.PNG" alt="Home" />
<h3 align="center">Picture-4</h3>

In [23]:
# Some Data Analysis
df.groupby(by="Company size", dropna=False).count()

,Name,Category,Website,Phone,Headquarters,Founded,Specialties
Company size,,,,,,,
0-1 employees,2,2,2,0,0,0,0
"1,001-5,000 employees",3,3,3,0,1,2,2
11-50 employees,41,41,40,15,30,34,29
2-10 employees,26,26,24,8,9,20,14
201-500 employees,10,10,10,5,8,8,9
"501-1,000 employees",4,4,4,2,3,3,3
51-200 employees,23,23,23,9,17,22,20
NaN,1,1,0,0,1,0,0


In [59]:
#Companies located on Dubai
df[df.Headquarters.str.contains("Dubai").fillna(False)]

,Name,Category,Website,Phone,Company size,Headquarters,Founded,Specialties
3,Infracom Communication Networks,Networks,http://infracommunication.com/,NaN,11-50 employees,"Jebel Ali, Dubai",2008,"TELECOMMUNICATION INFRASTRUCTURE, SECURITY SUR..."
23,Spectrum Networks,Networks,http://www.specnt.com/,NaN,51-200 employees,"JLT, Dubai",2004,"IT Trainings, Authorized Training Partners, La..."
24,New Trend Computer Networks L.L.C,Networks,http://www.newtrend.ae/,+97143931145\nPhone number is +97143931145,51-200 employees,"Dubai, Dubai",1998,"Information Technology, Network Solutions , IT..."
41,GWC Networks LLC,Networks,http://www.gwcnetworks.com,NaN,51-200 employees,"Dubai, Dubai",2004,"Network Performance Monitoring, Management & T..."
44,Abrus Networks,Networks,http://www.abrusnetworks.com,NaN,11-50 employees,"Business Bay, Dubai",2014,"Technology Infrastructure, Cloud Computing, AW..."
46,Zoomline Networks Technology,Networks,https://zoomlinenetwork.com,0553786444\nPhone number is 0553786444,11-50 employees,"Dubai, UAE",2008,"Datacentre, Networking Solutions, Security Sol..."
48,Microsys Networks LLC | Structured Cabling Dubai,Networks,http://www.microsys.ae,NaN,11-50 employees,"Al Fahidi Street, Dubai",2010,"Structured Cabling Dubai, Network Cabling, and..."
49,Exclusive Networks ME Training,Networks,http://exclusive-networks.com/ae,NaN,201-500 employees,"Dubai, Dubai",NaN,NaN
53,MESH NETWORKS LLC,Networks,http://www.meshnetworks.ae,NaN,11-50 employees,"Business Bay, Dubai",NaN,"IT Infrastructure, Consultancy, Security, Auto..."
57,Microsys Networks LLC - ELV & ICT,Networks,https://www.microsys.ae/,04 271 6636\nPhone number is 04 271 6636,11-50 employees,"Port Saeed, Dubai",2011,ICT - Data Center Solutions - Storage Solution...
